# 上下文工程

[上下文工程](https://docs.langchain.com/oss/python/langchain/context-engineering)（Context Engineering）对于 Agent 得出正确的结果至关重要。模型回答不好，很多时候不是因为能力不足，而是因为没有获得足以推断出正确结果的上下文信息。通过上下文工程，增强 Agent 获取和管理上下文的能力，是很有必要的。

**LangGraph 将上下文分为三种类型：**

- 模型上下文（Model Context）
- 工具上下文（Tool Context）
- 生命周期上下文（Life-cycle Context）

无论哪种 Context，都需要定义它的 Schema。在这方面，LangGraph 提供了相当高的自由度，你可以使用 `dataclasses`、`pydantic`、`TypedDict` 这些包的任意一个创建你的 Context Schema.

In [1]:
# !pip install ipynbname

In [1]:
import os
import uuid
import sqlite3

from typing import Callable
from dotenv import load_dotenv
from dataclasses import dataclass
from langchain_openai import ChatOpenAI
from langchain.tools import tool, ToolRuntime
from langchain.agents import create_agent
from langchain.agents.middleware import dynamic_prompt, wrap_model_call, ModelRequest, ModelResponse, SummarizationMiddleware
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.store.memory import InMemoryStore
from langgraph.store.sqlite import SqliteStore

# 加载模型配置
_ = load_dotenv()

# 加载模型
llm = ChatOpenAI(
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url=os.getenv("DASHSCOPE_BASE_URL"),
    model="Qwen/Qwen3-8B",
    temperature=0.7,
)

## 一、动态修改系统提示词

上下文工程与前序章节的中间件（middleware）和记忆（memory）密不可分。上下文的具体实现依赖中间件，而上下文的存储则依赖记忆系统。具体来讲，LangGraph 预置了 `@dynamic_prompt` 中间件，用于动态修改系统提示词。

既然是动态修改，肯定需要某个条件来触发修改。除了开发触发逻辑，我们还需要从智能体中获取触发逻辑所需的即时变量。这些变量通常存储在以下三个存储介质中：

- 运行时（Runtime）- 所有节点共享一个 Runtime。同一时刻，所有节点取到的 Runtime 的值是相同的。一般用于存储时效性要求较高的信息。
- 短期记忆（State）- 在节点之间按顺序传递，每个节点接收上一个节点处理后的 State。主要用于存储 Prompt 和 AI Message。
- 长期记忆（Store）- 负责持久化存储，可以跨 Workflow / Agent 保存信息。可以用来存用户偏好、以前算过的统计值等。

以下三个例子，分别演示如何使用来自 Runtime、State、Store 中的上下文，编写触发条件。

### 1）使用 `State` 加载上下文

利用 `State` 中蕴含的信息操纵 system prompt.

In [6]:
@dynamic_prompt
def state_aware_prompt(request: ModelRequest) -> str:
    # request.messages is a shortcut for request.state["messages"]
    message_count = len(request.messages)

    base = "You are a helpful assistant."

    if message_count > 6:
        base += "\nThis is a long conversation - be extra concise."

    # 临时打印base看效果
    print(base)

    return base

agent = create_agent(
    model=llm,
    middleware=[state_aware_prompt]
)

result = agent.invoke(
    {"messages": [
        {"role": "user", "content": "广州今天的天气怎么样？"},
        {"role": "assistant", "content": "广州天气很好"},
        {"role": "user", "content": "吃点什么好呢"},
        {"role": "assistant", "content": "要不要吃香茅鳗鱼煲"},
        {"role": "user", "content": "香茅是什么"},
        {"role": "assistant", "content": "香茅又名柠檬草，常见于泰式冬阴功汤、越南烤肉"},
        {"role": "user", "content": "auv 那还等什么，咱吃去吧"},
    ]},
)

for message in result['messages']:
    message.pretty_print()

You are a helpful assistant.
This is a long conversation - be extra concise.
================================ Human Message =================================

广州今天的天气怎么样？
================================== Ai Message ==================================

广州天气很好
================================ Human Message =================================

吃点什么好呢
================================== Ai Message ==================================

要不要吃香茅鳗鱼煲
================================ Human Message =================================

香茅是什么
================================== Ai Message ==================================

香茅又名柠檬草，常见于泰式冬阴功汤、越南烤肉
================================ Human Message =================================

auv 那还等什么，咱吃去吧
================================== Ai Message ==================================

那走吧，正好去尝尝地道的香茅鳗鱼煲！


把 `message_count > 6` 里的 6 改成 7，试试看会发生什么。

### 2）使用 `Store` 加载上下文

In [7]:
@dataclass
class Context:
    user_id: str

@dynamic_prompt
def store_aware_prompt(request: ModelRequest) -> str:
    user_id = request.runtime.context.user_id

    # Read from Store: get user preferences
    store = request.runtime.store
    user_prefs = store.get(("preferences",), user_id)

    base = "You are a helpful assistant."

    if user_prefs:
        style = user_prefs.value.get("communication_style", "balanced")
        base += f"\nUser prefers {style} responses."

    return base

store = InMemoryStore()

agent = create_agent(
    model=llm,
    middleware=[store_aware_prompt],
    context_schema=Context,
    store=store,
)

# 预置两条偏好信息
store.put(("preferences",), "user_1", {"communication_style": "Chinese"})
store.put(("preferences",), "user_2", {"communication_style": "Korean"})

In [10]:
# 用户1喜欢中文回复
result = agent.invoke(
    {"messages": [
        {"role": "system", "content": "You are a helpful assistant. Please be extra concise."},
        {"role": "user", "content": 'What is a "hold short line"?'}
    ]},
    context=Context(user_id="user_1"),
)

for message in result['messages']:
    message.pretty_print()

================================ System Message ================================

You are a helpful assistant. Please be extra concise.
================================ Human Message =================================

What is a "hold short line"?
================================== Ai Message ==================================

“Hold short line” 是航空术语，指飞机在跑道上等待起飞时应停在的标记线，通常位于跑道起始端的某个位置，用于确保飞机在起飞前保持适当的安全距离。


In [11]:
# 用户2喜欢韩文回复
result = agent.invoke(
    {"messages": [
        {"role": "system", "content": "You are a helpful assistant. Please be extra concise."},
        {"role": "user", "content": 'What is a "hold short line"?'}
    ]},
    context=Context(user_id="user_2"),
)

for message in result['messages']:
    message.pretty_print()

================================ System Message ================================

You are a helpful assistant. Please be extra concise.
================================ Human Message =================================

What is a "hold short line"?
================================== Ai Message ==================================

"_hold short line_"은 항공 분야에서 사용되는 용어로, 특정 지점에 비행기가 멈춰 있을 수 있도록 지정된 선을 의미합니다. 일반적으로 이선은 활주로의 끝 부분과 이륙 지점 사이에 위치하며, 항공기의 이륙 준비 상태를 나타냅니다.


### 3）使用 `Runtime` 加载上下文

In [12]:
@dataclass
class Context:
    user_role: str
    deployment_env: str

@dynamic_prompt
def context_aware_prompt(request: ModelRequest) -> str:
    # Read from Runtime Context: user role and environment
    user_role = request.runtime.context.user_role
    env = request.runtime.context.deployment_env

    base = "You are a helpful assistant."

    if user_role == "admin":
        base += "\nYou can use the get_weather tool."
    else:
        base += "\nYou are prohibited from using the get_weather tool."

    if env == "production":
        base += "\nBe extra careful with any data modifications."

    return base

@tool
def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

agent = create_agent(
    model=llm,
    tools=[get_weather],
    middleware=[context_aware_prompt],
    context_schema=Context,
    checkpointer=InMemorySaver(),
)

In [13]:
# 利用 Runtime 中的两个变量，动态控制 System prompt
# 将 user_role 设为 admin，允许使用天气查询工具
config = {'configurable': {'thread_id': str(uuid.uuid4())}}
result = agent.invoke(
    {"messages": [{"role": "user", "content": "广州今天的天气怎么样？"}]},
    context=Context(user_role="admin", deployment_env="production"),
    config=config,
)

for message in result['messages']:
    message.pretty_print()

================================ Human Message =================================

广州今天的天气怎么样？
================================== Ai Message ==================================
Tool Calls:
  get_weather (019ac8e704e6df19d3769b303f7e390e)
 Call ID: 019ac8e704e6df19d3769b303f7e390e
  Args:
    city: 广州
================================= Tool Message =================================
Name: get_weather

It's always sunny in 广州!
================================== Ai Message ==================================

广州今天天气晴朗，阳光明媚！记得做好防晒哦！


In [14]:
# 若将 user_role 改为 viewer，则无法使用天气查询工具
config = {'configurable': {'thread_id': str(uuid.uuid4())}}
result = agent.invoke(
    {"messages": [{"role": "user", "content": "广州今天的天气怎么样？"}]},
    context=Context(user_role="viewer", deployment_env="production"),
    config=config,
)

for message in result['messages']:
    message.pretty_print()

================================ Human Message =================================

广州今天的天气怎么样？
================================== Ai Message ==================================
Tool Calls:
  get_weather (019ac8e718f72c910f14732491e266e7)
 Call ID: 019ac8e718f72c910f14732491e266e7
  Args:
    city: 广州
================================= Tool Message =================================
Name: get_weather

It's always sunny in 广州!
================================== Ai Message ==================================

广州今天天气晴朗，阳光明媚，是个非常适合外出的好日子！记得做好防晒措施哦。


In [15]:
result['messages']

[HumanMessage(content='广州今天的天气怎么样？', additional_kwargs={}, response_metadata={}, id='d52217de-4940-45fa-bbfb-e8bb89a94e13'),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 183, 'total_tokens': 202, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'Qwen/Qwen3-8B', 'system_fingerprint': '', 'id': '019ac8e712900596018ffade98d5e806', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--5117421b-c3f6-4750-9ec4-26b8141ba8d4-0', tool_calls=[{'name': 'get_weather', 'args': {'city': '广州'}, 'id': '019ac8e718f72c910f14732491e266e7', 'type': 'tool_call'}], usage_metadata={'input_tokens': 183, 'output_tokens': 19, 'total_tokens': 202, 'input_token_details': {}, 'output_token_details': {'reasoning': 0}}),
 ToolMessage(content="It's 

## 二、动态修改消息列表

LangGraph 预制了动态修改消息列表（Messages）的中间件 `@wrap_model_call`。上一节已经演示如何从 `State`、`Store`、`Runtime` 中获取上下文，本节将不再一一演示。在下面这个例子中，我们主要演示如何使用 `Runtime` 将本地文件的内容注入消息列表。

In [16]:
@dataclass
class FileContext:
    uploaded_files: list[dict]

@wrap_model_call
def inject_file_context(
    request: ModelRequest,
    handler: Callable[[ModelRequest], ModelResponse]
) -> ModelResponse:
    """Inject context about files user has uploaded this session."""
    uploaded_files = request.runtime.context.uploaded_files

    try:
        base_dir = os.path.dirname(os.path.abspath(__file__))
    except Exception as e:
        import ipynbname
        import os
        notebook_path = ipynbname.path()
        base_dir = os.path.dirname(notebook_path)

    file_sections = []
    for file in uploaded_files:
        name, ftype = "", ""
        path = file.get("path")
        if path:
            base_filename = os.path.basename(path)
            stem, ext = os.path.splitext(base_filename)
            name = stem or base_filename
            ftype = (ext.lstrip(".") if ext else None)

            # 构建文件描述内容
            content_list = [f"名称: {name}"]
            if ftype:
                content_list.append(f"类型: {ftype}")

            # 解析相对路径为绝对路径
            abs_path = path if os.path.isabs(path) else os.path.join(base_dir, path)

            # 读取文件内容
            content_block = ""
            if abs_path and os.path.exists(abs_path):
                try:
                    with open(abs_path, "r", encoding="utf-8") as f:
                        content_block = f.read()
                except Exception as e:
                    content_block = f"[读取文件错误 '{abs_path}': {e}]"
            else:
                content_block = "[文件路径缺失或未找到]"

            section = (
                f"---\n"
                f"{chr(10).join(content_list)}\n\n"
                f"{content_block}\n"
                f"---"
            )
            file_sections.append(section)

        file_context = (
            "已加载的会话文件：\n"
            f"{chr(10).join(file_sections)}"
            "\n回答问题时请参考这些文件。"
        )

        # Inject file context before recent messages
        messages = [  
            *request.messages,
            {"role": "user", "content": file_context},
        ]
        request = request.override(messages=messages)  

    return handler(request)

agent = create_agent(
    model=llm,
    middleware=[inject_file_context],
    context_schema=FileContext,
)

In [17]:
result = agent.invoke(
    {
        "messages": [{
            "role": "user",
            "content": "关于上海地铁的无脸乘客，有什么需要注意的？",
        }],
    },
    context=FileContext(uploaded_files=[{"path": "./docs/rule_horror.md"}]),
)

for message in result['messages']:
    message.pretty_print()

================================ Human Message =================================

关于上海地铁的无脸乘客，有什么需要注意的？
================================== Ai Message ==================================

欢迎来到上海的夜色中。关于“无脸乘客”，请务必牢记以下注意事项：

**1. 地铁运营结束后的警惕：**
- 上海地铁通常在深夜23:00左右停止运营，若你未能及时下车，且仍在车厢内，可能会遇到一位无脸乘客。
- 他会出现并低声问你：“你要去哪？”
- **请务必报出一个真实存在的上海地名**，例如“人民广场”、“陆家嘴”、“徐家汇”等。
- **切勿说出不存在的地名**，也切勿保持沉默，否则你可能会在车厢内看到自己的尸体。

**2. 与无脸乘客互动的禁忌：**
- 无脸乘客不会说话，也不会有面部表情，他的存在是无形的。
- **即使他问你问题，也请不要试图与他交流**，更不要试图看清他的脸。
- **回答时要简短、明确，只说出一个地名**，并迅速离开车厢。

**3. 保持冷静与理智：**
- 无脸乘客的出现可能是某种超自然现象，也可能是心理暗示或幻觉。
- 如果你感到不安或怀疑，**请立即离开地铁站，不要继续留在车厢内**。
- **不要相信任何与无脸乘客有关的异常现象**，比如他试图引导你去某个地方、或对你做出其他异常举动。

**4. 提高安全意识：**
- 如果你已经离开地铁站，但仍然感到不安，建议你**不要独自在深夜外出**，尤其是前往偏僻或人烟稀少的区域。
- 保持手机电量充足，随时可以联系他人或寻求帮助。

**5. 信任自己的直觉：**
- 如果你感觉无脸乘客是危险的，**请相信自己的直觉，迅速离开现场**。
- 无脸乘客可能只是传说，但在某些文化背景下，它被认为是一种警示或禁忌。

总之，在上海的深夜地铁中，遇到无脸乘客时，**保持冷静、理智，遵守规则，是保护自己的关键**。希望你在夜色中平安无事。


## 三、在工具中使用上下文

下面，我们尝试在工具中使用存储在 `SqliteStore` 中的上下文信息。

In [25]:
# 删除SQLite数据库
if os.path.exists("user-info.db"):
    os.remove("user-info.db")

# 创建SQLite存储
conn = sqlite3.connect("user-info.db", check_same_thread=False, isolation_level=None)
conn.execute("PRAGMA journal_mode=WAL;")
conn.execute("PRAGMA busy_timeout = 30000;")

store = SqliteStore(conn)

print(store.search("user_info"))

# 预置两条用户信息
store.put(("user_info",), "柳如烟", {"description": "清冷才女，身怀绝技，为寻身世之谜踏入江湖。", "birthplace": "吴兴县"})
store.put(("user_info",), "苏慕白", {"description": "孤傲剑客，剑法超群，背负家族血仇，隐于市井追寻真相。", "birthplace": "杭县"})

[]


### 1）基础用例

使用 `ToolRuntime`

In [26]:
@tool
def fetch_user_data(
    user_id: str,
    runtime: ToolRuntime
) -> str:
    """
    Fetch user information from the in-memory store.

    :param user_id: The unique identifier of the user.
    :param runtime: The tool runtime context injected by the framework.
    :return: The user's description string if found; an empty string otherwise.
    """
    store = runtime.store
    user_info = store.get(("user_info",), user_id)

    user_desc = ""
    if user_info:
        user_desc = user_info.value.get("description", "")

    return user_desc

agent = create_agent(
    model=llm,
    tools=[fetch_user_data],
    store=store,
)

In [27]:
result = agent.invoke({
    "messages": [{
        "role": "user",
        "content": "五分钟之内，我要柳如烟的全部信息"
    }]
})

for message in result['messages']:
    message.pretty_print()

================================ Human Message =================================

五分钟之内，我要柳如烟的全部信息
================================== Ai Message ==================================
Tool Calls:
  fetch_user_data (019ac8f01f1db1b9fbf6f85c8b49fc93)
 Call ID: 019ac8f01f1db1b9fbf6f85c8b49fc93
  Args:
    user_id: 柳如烟
================================= Tool Message =================================
Name: fetch_user_data

清冷才女，身怀绝技，为寻身世之谜踏入江湖。
================================== Ai Message ==================================

柳如烟是一位清冷才女，身怀绝技，性格孤傲，行事果断。她因追寻自己的身世之谜而踏入江湖，历经种种磨难与挑战，逐渐揭开身世背后的秘密。在江湖中，她以智慧和武艺著称，常常以一个旁观者的姿态观察世事，却在关键时刻挺身而出，为正义而战。她的故事充满了悬疑与冒险，令人回味无穷。


### 2）复杂一点的例子

使用 `ToolRuntime[Context]`

In [28]:
@dataclass
class Context:
    key: str

@tool
def fetch_user_data(
    user_id: str,
    runtime: ToolRuntime[Context]
) -> str:
    """
    Fetch user information from the in-memory store.

    :param user_id: The unique identifier of the user.
    :param runtime: The tool runtime context injected by the framework.
    :return: The user's description string if found; an empty string otherwise.
    """
    key = runtime.context.key

    store = runtime.store
    user_info = store.get(("user_info",), user_id)

    user_desc = ""
    if user_info:
        user_desc = user_info.value.get(key, "")

    return f"{key}: {user_desc}"

agent = create_agent(
    model=llm,
    tools=[fetch_user_data],
    store=store,
)

In [29]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "五分钟之内，我要柳如烟的全部信息"}]},
    context=Context(key="birthplace"),
)

for message in result['messages']:
    message.pretty_print()

================================ Human Message =================================

五分钟之内，我要柳如烟的全部信息
================================== Ai Message ==================================
Tool Calls:
  fetch_user_data (019ac8f0733ffa7402952cad57c67139)
 Call ID: 019ac8f0733ffa7402952cad57c67139
  Args:
    user_id: 柳如烟
================================= Tool Message =================================
Name: fetch_user_data

birthplace: 吴兴县
================================== Ai Message ==================================

柳如烟的出生地是吴兴县。如果您需要更多关于她的信息，请进一步说明。


### 四、压缩上下文

LangChain 提供了内置的中间件 `SummarizationMiddleware` 用于压缩上下文。该中间件维护的是典型的 **生命周期上下文**，与 **模型上下文** 和 **工具上下文** 的瞬态更新不同，生命周期上下文会持续更新：持续将旧消息替换为摘要。

除非上下文超长，导致模型能力降低，否则不需要使用 `SummarizationMiddleware`。一般来说，触发摘要得值可以设得较大。比如：

- `max_tokens_before_summary`: 3000
- `messages_to_keep`: 20

> 如果你想了解更多关于上下文腐坏（Context Rot）的信息，Chroma 团队在 2025 年 7 月 14 日发布的 [*Context Rot: How Increasing Input Tokens Impacts LLM Performance*](https://research.trychroma.com/context-rot)，系统性地揭示了长上下文导致模型性能退化的现象。

In [32]:
# 创建短期记忆
checkpointer = InMemorySaver()

# 创建带内置摘要中间件的Agent
# 为了让配置能在我们的例子里生效，这里的触发值设得很小
agent = create_agent(
    model=llm,
    middleware=[
        SummarizationMiddleware(
            model=llm,
            trigger=('tokens', 40),  # Trigger summarization at 40 tokens
            keep=("messages",1),  # Keep last 1 messages after summary
        ),
    ],
)

In [33]:
result = agent.invoke(
    {"messages": [
        {"role": "user", "content": "广州今天的天气怎么样？"},
        {"role": "assistant", "content": "广州天气很好"},
        {"role": "user", "content": "吃点什么好呢"},
        {"role": "assistant", "content": "要不要吃香茅鳗鱼煲"},
        {"role": "user", "content": "香茅是什么"},
        {"role": "assistant", "content": "香茅又名柠檬草，常见于泰式冬阴功汤、越南烤肉"},
        {"role": "user", "content": "auv 那还等什么，咱吃去吧"},
    ]},
    checkpointer=checkpointer,
)

for message in result['messages']:
    message.pretty_print()

================================ Human Message =================================

Here is a summary of the conversation to date:

广州今天的天气很好。香茅又名柠檬草，常见于泰式冬阴功汤、越南烤肉。
================================ Human Message =================================

auv 那还等什么，咱吃去吧
================================== Ai Message ==================================

哈哈，你说得对！广州天气这么好，正好是出门吃美食的好时机。香茅作为泰式冬阴功汤和越南烤肉的常见香料，它的清新香气真的能让食物更上一层楼。要不要一起去尝尝地道的泰式冬阴功或者越南烤肉？我可以推荐一些当地不错的餐馆哦！
